In [55]:
from google.cloud import videointelligence_v1p3beta1
from google.cloud.videointelligence_v1p3beta1 import enums
from google.cloud import storage
from google.oauth2 import service_account
import os
from collections import defaultdict

In [4]:
CRED_DIR = 'creds'
VIDEO_BUCKET = 'margot'

In [5]:
credentials = service_account.Credentials.from_service_account_file(os.path.join(CRED_DIR, 'margot-ab6e92107352.json'))

In [6]:
client_gs = storage.Client(project='margot', credentials=credentials)

In [51]:
video_list = [b.name for b in client_gs.get_bucket(VIDEO_BUCKET).list_blobs()]

In [7]:
client_vi = videointelligence_v1p3beta1.VideoIntelligenceServiceClient(credentials=credentials)

In [52]:
features = [videointelligence_v1p3beta1.enums.Feature.SPEECH_TRANSCRIPTION]

In [53]:
config = videointelligence_v1p3beta1.types.SpeechTranscriptionConfig(
                language_code='en-US',
                enable_automatic_punctuation=True)

In [54]:
video_context = videointelligence_v1p3beta1.types.VideoContext(
                    speech_transcription_config=config)

In [56]:
margot_texts = defaultdict()

for v in video_list[:2]:
    
    print(f'annotating {v}...')

    res = client_vi.annotate_video(
        input_uri=f'gs://{VIDEO_BUCKET}/{v}',
        features=features,
        video_context=video_context).result()
    
    texts = ''
    
    for _ in res.annotation_results[0].speech_transcriptions:
        try:
            texts += _.alternatives[0].transcript
        except:
            pass
    
    margot_texts[v] = texts

annotating 73 Questions With Margot Robbie _ Vogue-Bl5630CeYFs.mp4...
annotating Guest Host Chris Pratt Interviews Margot Robbie-KjICBHj1EzM.webm...


In [57]:
margot_texts

defaultdict(None,
            {'73 Questions With Margot Robbie _ Vogue-Bl5630CeYFs.mp4': "Hey Margo. Hi, so excited to be here at this beautiful Cottage asking you 73 questions. Yes, of course, come on in. Thank you. So, how did you start your day today? I had a nice cup of tea and I went for a swim. What do you have planned for the rest of the day having some friends over for brunch you could spend the day with anyone in the world? Who would that be JK Rowling? And what would you do? I would ask her 73 questions. No way. Yeah, I'd probably ask you like a million in 73 questions. But yeah, what's one thing that you would love to master? I'd really like to be good at pool. Are you a texter or a collar? I'm Nicola who was the last person that you call my cousin Julia? What'd you guys talk about? I asked her to bring some things over for brunch. Nice. Do you want a water or something callosum? Thank you. So you play piano? No, that's just kind of a impressive is the Australian national a

In [ ]:
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer